# Werewolf Simulator
The following runs a simulation of the game Werewolf with multiple agents, it is built on top of and uses the Agentscope library, documentation can be found here https://doc.agentscope.io/, and the open source code that we extend on is here https://github.com/modelscope/agentscope/tree/main.
### Rules of Werewolf
- There are 4 roles, the Werewolves, Villagers, Witch, and Seer (Witch and Seer are on the Villager team)
- Each night:
    - The Werewolves discuss and vote on a player to eliminate
    - The Witch is told what player the wolves voted to eliminate, and is given the choice to use their potion of healing to reserruct the eliminated player, or use their poison to eliminate another player. Note that each power can only be used once in a game
    - The seer can pick any other player and find out what their role is (one player per night)
- After the events of the night, all surviving players discuss amongst themselves and vote on a player to eliminate
- Werewolves win if their numbers equal or exceed Villagers.
- Villagers win if all Werewolves are eliminated.

In [235]:
#%pip install -r requirements.txt

In [236]:
import os
from dotenv import load_dotenv

from typing import Optional, Union, Sequence, Any, Dict
from functools import partial
import openai  
import faiss
import numpy as np
import json
import random 
import logging
import csv
from datetime import datetime
from zoneinfo import ZoneInfo
import sys
import time
from typing import Callable, Any

import numpy as np
from loguru import logger

from agentscope.parsers.json_object_parser import MarkdownJsonDictParser
from agentscope.parsers import ParserBase
from agentscope.message import Msg
from agentscope.msghub import msghub
from agentscope.agents import AgentBase
from agentscope.memory.temporary_memory import TemporaryMemory
from agentscope.pipelines.functional import sequentialpipeline
import agentscope

from utils.werewolf_utils import (
    extract_name_and_id,
    n2s,
    set_parsers,
    generate_model_response
)

# Configure the Logger

In [237]:
# Default log configuration
LOG_LEVEL = logging.INFO
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
DATE_FORMAT = "%Y-%m-%d %H:%M:%S"

# Initial logging setup
logging.basicConfig(
    filename=None,
    level=LOG_LEVEL,
    format=LOG_FORMAT,
    datefmt=DATE_FORMAT
)

logger = logging.getLogger(__name__)
structured_logger = None

def change_log_file(new_filename: str):
    """
    Updates the logger handler to point to a different log file.
    """
    
    # Ensure the directory exists
    log_dir = os.path.dirname(new_filename)
    if not os.path.exists(log_dir):
        os.makedirs(log_dir, exist_ok=True)  # Create directories if they don't exist

    # Remove all old handlers
    for handler in logger.handlers[:]:
        logger.removeHandler(handler)
        handler.close()

    # Create a new file handler
    new_handler = logging.FileHandler(new_filename)
    new_handler.setLevel(LOG_LEVEL)  # Reset level
    new_handler.setFormatter(logging.Formatter(LOG_FORMAT, datefmt=DATE_FORMAT))  # Reset formatter

    # Add new handler to logger
    logger.addHandler(new_handler)

def log_params(log_filepath: str, **kwargs) -> None:
    """
    Writes all parameters passed to the function into a JSON file.
    
    Args:
        log_filepath (str): Path where the JSON file will be written.
        **kwargs: Any number of keyword arguments representing parameter names and values.
    """
    with open(log_filepath, "w") as file:
        json.dump(kwargs, file, separators=(",", ":"))  # Compact JSON
        file.write("\n")  # Ensures a newline at the end

    print(f"Parameters written to {log_filepath}")

def log_vectorstore_retrieval(query: str, similar_documents: str):
    """Logs the most similar documents from vectorstore"""

    logger.info("===")
    logger.info(f"Query: {query}")
    logger.info(f"Retrieved similar documents: {similar_documents}")
    logger.info("===")

    structured_logger.log("vectorstore_retrieve", similar_documents)

def log_message(msg: Msg):
    """Logs a message object""" 

    logger.info(msg.name + ": " + msg.content)
    
# Logging utilities
class StructuredLogger:
    """
    A logger that outputs to TSV files for structured logging of LLM interactions.
    """

    # Define all possible message types for documentation
    MESSAGE_TYPES = [
        "vectorstore_add",           # Adding a case to vectorstore
        "vectorstore_retrieve",      # Retrieving similar cases
        "vectorstore_summarize",     # Summarizing cached decisions
        "argument_prompt",           # Input prompt for pro arguments
        "argument_response",         # Response from pro argument LLM
        "judge_prompt",              # Input prompt for judge
        "judge_response",            # Response from judge LLM
        "fast_prompt",               # Input prompt for fast LLM
        "fast_response",             # Fast LLM respnose
        "slow_prompt",               # Input prompt for slow LLM
        "slow_response",             # Slow LLM response

        # Werewolf-specific messages
        "game_init",                 # Game initialization config
        "game_state",                # Current state of the game
        "phase"                      # Current phase of game (day/night)
    ]

    def __init__(self, tsv_filepath: str):
        """
        Initialize the logger with a base filepath.

        Args:
            tsv_filepath: Filepath for TSV log files (includes timestamp)
        """
        self.tsv_filepath = tsv_filepath

        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(self.tsv_filepath), exist_ok=True)

        # Initialize TSV file with headers if it doesn't exist
        if not os.path.exists(self.tsv_filepath):
            with open(self.tsv_filepath, "w", newline="", encoding="utf-8") as tsvfile:
                writer = csv.writer(tsvfile, delimiter="\t")
                writer.writerow(["timestamp", "message_type", "message"])

    def log(self, message_type: str, message_data: Union[str, Dict[str, Any], list[Any], Msg]) -> None:
        """
        Log a message to the TSV file.

        Args:
            message_type: Type of message (one of MESSAGE_TYPES)
            message_data: Data to be logged (will be converted to JSON if not already a string)
        """
        if message_type not in self.MESSAGE_TYPES:
            raise ValueError(
                f"Invalid message_type: {message_type}. Must be one of {self.MESSAGE_TYPES}"
            )

        # Convert message_data to string if it's not already
        if isinstance(message_data, (dict, list)):
            message = json.dumps(message_data, ensure_ascii=False)
        else:
            message = str(message_data)

        timestamp = datetime.now().isoformat()

        # Append to TSV file
        with open(self.tsv_filepath, "a", newline="", encoding="utf-8") as tsvfile:
            writer = csv.writer(tsvfile, delimiter="\t", quotechar='`', quoting=csv.QUOTE_MINIMAL)
            writer.writerow([timestamp, message_type, message])

## Vector Store Implementations
Vector store classes 

In [238]:
class ReflectiveVectorstoreMemory:
    """Reflective Vectorstore-based memory using FAISS and OpenAI embeddings."""
    
    def __init__(self, embedding_model: str = "text-embedding-ada-002"):
        """
        Initialize the vectorstore with FAISS and OpenAI embeddings.
        
        Args:
            embedding_model (str): The OpenAI embedding model to use.
        """
        self.embedding_model = embedding_model
        # Vector Store
        self.index = faiss.IndexFlatL2(1536)  # 1536 is the dimensionality of 'text-embedding-ada-002'
        self.messages = []  # To store actual messages (content)
        self.summaries = []  # To store summaries

    def _get_embedding(self, text: str) -> np.ndarray:
        """
        Generate an embedding for the given text using OpenAI.
        
        Args:
            text (str): The input text to embed.
        
        Returns:
            np.ndarray: The embedding vector as a NumPy array.
        """
        client = openai.OpenAI()

        response = retry_with_fallback(lambda: client.embeddings.create(
            input=text,
            model=self.embedding_model
        ))
        
        return np.array(response.data[0].embedding, dtype="float32")

    def add_message(self, message: Union[Msg, Sequence[Msg]]):
        """Add a message to the FAISS index."""
        if not isinstance(message, list):
            message = [message]
        for msg in message:
            self.messages.append(msg.name + ": " + msg.content)
        
        structured_logger.log("vectorstore_add", convert_messages_to_string(message))
        
    def summarize_cycle(self, survivors: list, secondary_model: str, cycle_type: str = "day"):
        """
        Generate a summary of the conversation after a day/night cycle,
        and add it to the vector store.
        
        Args:
            survivors (list): List of current alive players.
            secondary_model (str): Name of secondary model.
            cycle_type (str): The type of cycle ("day" or "night").
        """
        client = openai.OpenAI()
        
        # Combine all messages since the last summary
        history = "\n".join(self.messages)

        # Use retry_with_fallback with a lambda function
        secondary_model_response = retry_with_fallback(
            lambda: generate_model_response(
                model=secondary_model,
                messages=[
                    {"role": "system", "content": "\n".join([ 
                        f"You are a strategic decision-maker reviewing past decisions in Werewolf.",
                        f"Survivors: {', '.join([survivor.name for survivor in survivors])}",
                        f"Summarize the decisions and reflect on the most important implications for future strategic decisions."
                    ])},
                    {"role": "user", "content": history},
                ]
            )
        )

        summary = secondary_model_response.text

        logger.info("===")
        logger.info(f"Input decisions: {history}")
        logger.info(f"Summarized decisions: {summary}")
        logger.info("===")

        structured_logger.log("vectorstore_summarize", {
            "input_decisions": history,
            "summary": summary 
        })

        # Embed the summary and add it to the vector store
        embedding = self._get_embedding(summary)
        self.index.add(np.array([embedding]))
        self.summaries.append(summary)

        # Clear messages for the next cycle
        self.messages.clear()

    def get_relevant_summaries_context(self, query: str, top_k: int = 1) -> str:
        """Retrieve the top-k most relevant summaries.
        
        Args:
            query (str): query to find similar summaries to.
            top_k (int): number of relevant summaries to retrieve.
        """
        if len(self.summaries) > 0:
            query_embedding = self._get_embedding(query)
            distances, indices = self.index.search(np.array([query_embedding]), top_k)
            results = [
                self.summaries[idx] for idx in indices[0] if idx < len(self.summaries)
            ]
            return "\n".join(list(set(results)))
        return ""
    
    def save_context(self, timestamp: str, game_num: int, player_role: str):
        """
        Save self.messages to a log file in the format:
        vectorstore/[timestamp]/[game_num]/[player_role].log

        Args:
            game_num (int): The game index.
            player_role (str): The role of the player.
        """

        # Construct the directory path
        dir_path = os.path.join("vectorstore", "courtroom", timestamp, str(game_num))

        # Ensure the directory exists
        os.makedirs(dir_path, exist_ok=True)

        # Construct the file path
        file_path = os.path.join(dir_path, f"{player_role}.log")

        # Save messages to the log file
        with open(file_path, "w", encoding="utf-8") as file:
            file.write("\n".join(self.summaries))

        print(f"Messages saved to {file_path}")

class VectorstoreMemory:
    """Vectorstore-based memory using FAISS and OpenAI embeddings."""
    
    def __init__(self, embedding_model: str = "text-embedding-ada-002"):
        """
        Initialize the vectorstore with FAISS and OpenAI embeddings.
        
        Args:
            embedding_model (str): The OpenAI embedding model to use.
        """
        self.embedding_model = embedding_model
        # Vector Store
        self.index = faiss.IndexFlatL2(1536)  # 1536 is the dimensionality of 'text-embedding-ada-002'
        self.messages = []  # To store actual messages (content)

    def _get_embedding(self, text: str) -> np.ndarray:
        """
        Generate an embedding for the given text using OpenAI.
        
        Args:
            text (str): The input text to embed.
        
        Returns:
            np.ndarray: The embedding vector as a NumPy array.
        """
        client = openai.OpenAI()
        response = retry_with_fallback(lambda: client.embeddings.create(
            input=text,
            model=self.embedding_model
        ))
        return np.array(response.data[0].embedding, dtype="float32")

    def add_message(self, message: Union[Msg, Sequence[Msg]]):
        """Add a message to the FAISS index."""
        if not isinstance(message, list):
            message = [message]

        structured_logger.log("vectorstore_add", convert_messages_to_string(message))
        # add embedding of message to vector store
        embeddings = [self._get_embedding(msg.name + ": " + msg.content) for msg in message]
        self.index.add(np.array(embeddings)) 
        self.messages.extend(message)

    def get_relevant_messages(self, query: str, top_k: int = 10) -> list:
        """Retrieve the top-k most relevant messages."""
        if not self.messages:
            return []
        query_embedding = self._get_embedding(query)
        distances, indices = self.index.search(np.array([query_embedding]), top_k)

        results = [
            self.messages[idx] for idx in indices[0] if idx < len(self.messages)
        ]
        return results

    def save_context(self, timestamp: str, game_num: int, player_role: str):
        """
        Save self.messages to a log file in the format:
        vectorstore/[timestamp]/[game_num]/[player_role].log

        Args:
            game_num (int): The game index.
            player_role (str): The role of the player.
        """

        # Construct the directory path
        dir_path = os.path.join("vectorstore", "courtroom", timestamp, str(game_num))

        # Ensure the directory exists
        os.makedirs(dir_path, exist_ok=True)

        # Construct the file path
        file_path = os.path.join(dir_path, f"{player_role}.log")

        # Save messages to the log file
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(convert_messages_to_string(self.messages))

        print(f"Messages saved to {file_path}")

        

## Custom Agents (edit here)
You can define custom Agents by inheriting from the AgentBase class like shown here.

Below is a copy of the DictDialogAgent that generates responses in a dict format that is compatible with our simulator. More documentation on AgentScope agents can be found here https://doc.agentscope.io/build_tutorial/builtin_agent.html, existing agent implementations can be found here https://github.com/modelscope/agentscope/tree/main/src/agentscope/agents.

For our case, modifying the agent class is, in conjunction with the parsers we pass in (more details in next cell), play a critical role in defining our agent behavior. We can edit this section by either defining a brand new agent type we want to explore, or modifying the current one. The primary (and mostly only) source of focus should be the reply the function in the agent class as that controls what an agent uses to generate a response, particular when we are working with different architectures, eg. how we use memory, single vs multi agent, we make those edits here.

In [239]:
class Lawyer(AgentBase):
    """An agent specializing in constructing LLM-based arguments to eliminate a specific target in the Werewolf game."""

    def __init__(
        self,
        name: str,
        sys_prompt: str,
        model_config_name: str,
        parser,
        context_messages: list,
        memory
    ) -> None:
        """
        Initialize the Lawyer agent.

        Args:
            name (str): Name of the Lawyer agent.
            sys_prompt (str): System prompt containing role instructions and context.
            model_config_name (str): Identifier for the model configuration that determines the LLM to be used.
            parser (ParserBase): Object responsible for formatting LLM outputs and extracting structured responses.
            context_messages (list): A list of prior messages providing context for the lawyer’s argumentation.
            memory: Memory management component used to store conversation history or additional data.
        """
        super().__init__(
            name=name,
            sys_prompt=sys_prompt,
            model_config_name=model_config_name,
        )
        self.parser = parser
        self.context_messages = context_messages
        self.memory = memory
        self.model_config_name = model_config_name

    def argue(self, available_targets: list) -> dict:
        """
        Generate a rationale for eliminating a particular player in the Werewolf game.

        Args:
            game_state (dict): The current state of the game (e.g., who is alive, who is dead).
            available_targets (list): List of potential targets that can be selected for elimination.

        Returns:
            dict:
                A dictionary containing:
                  - "target" (str): The chosen player to eliminate.
                  - "argument" (str): The Lawyer’s supporting reasoning.
        """
        if not available_targets:
            return {"target": None, "argument": "No valid targets available."}
        
        target = random.choice(available_targets) # Select random a target to argue for

        # Construct prompt for argument generation
        lawyer_prompt_with_target = Prompts.lawyer_prompt.format(target)

        # Prepare the prompt for the model
        formatted_prompt = self.model.format(
            Msg("system", self.sys_prompt, role="system"),
            Msg("system", lawyer_prompt_with_target, role="system"),
            self.context_messages and Msg("system", f"Relevant messages from memory: {convert_messages_to_string(self.context_messages)}", role="system"),
            Msg("system", self.parser.format_instruction, role="system"),
        )

        structured_logger.log("argument_prompt", f"{formatted_prompt[0]["content"]}\n{formatted_prompt[1]["content"]}\n{formatted_prompt[2]["content"]}")

        # Call the LLM to generate an argument
        raw_response = retry_with_fallback(
            lambda: generate_model_response(self.model_config_name, formatted_prompt)
        )

        parsed_response = self.parser.parse(raw_response).parsed

        return {
            "target": target,
            "argument": parsed_response
        }

class Judge(AgentBase):
    """An agent that evaluates competing arguments and selects who should be eliminated in the Werewolf game."""

    def __init__(
        self,
        name: str,
        sys_prompt: str,
        model_config_name: str,
        parser,
        context_messages: list,
        memory
    ) -> None:
        """
        Initialize the Judge agent.

        Args:
            name (str): Name of the Judge agent.
            sys_prompt (str): System prompt containing role instructions and context.
            model_config_name (str): Identifier for the model configuration that determines the LLM to be used.
            parser (ParserBase): Object responsible for formatting LLM outputs and extracting structured responses.
            context_messages (list): List of prior messages providing context for the Judge’s decision-making.
            memory: Memory component used to store and retrieve conversation or historical data.
        """
        super().__init__(
            name=name,
            sys_prompt=sys_prompt,
            model_config_name=model_config_name,
        )
        self.parser = parser
        self.context_messages = context_messages
        self.memory = memory
        self.model_config_name = model_config_name

    def decide(self, game_state: dict, argument_1: dict, argument_2: dict):
        """
        Evaluate two lawyers' arguments and choose a target for elimination.

        Args:
            game_state (dict): The current state of the Werewolf game (e.g., which players are alive or dead).
            argument_1 (dict):
                First lawyer’s argument dictionary with:
                  - "target" (str): Proposed player to eliminate.
                  - "argument" (str): Reasoning for that choice.
            argument_2 (dict): Second lawyer’s argument dictionary with the same keys as argument_1.

        Returns:
            The raw response from the model, containing the Judge’s final decision.
        """
        target_1 = argument_1["target"]
        argument_1 = argument_1["argument"]
        
        target_2 = argument_2["target"]
        argument_2 = argument_2["argument"]

        # Construct formatted judge prompt
        formatted_judge_prompt = Prompts.judge_prompt.format(game_state["survivors"], game_state["dead"], game_state["werewolves"], target_1, argument_1, target_2, argument_2)

        # Prepare the prompt for the model
        formatted_prompt = self.model.format(
            Msg("system", self.sys_prompt, role="system"),
            Msg("system", formatted_judge_prompt, role="system"),
            Msg("system", self.parser.format_instruction, "system"),
        )

        structured_logger.log("judge_prompt", f"{formatted_prompt[0]["content"]}\n{formatted_prompt[1]["content"]}")

        raw_response = retry_with_fallback(
            lambda: generate_model_response(self.model_config_name, formatted_prompt)
        )

        return raw_response

class WerewolfAgent(AgentBase):
    """
    Represents a "werewolf" in the game, producing responses in dict form.

    Optionally uses reflective memory before retrieving context from a FAISS vectorstore.
    Contains parsing capabilities to ensure structured outputs.
    """

    def __init__(
        self,
        name: str,
        sys_prompt: str,
        model_config_name: str,
        reflect_before_vectorstore: bool,
        similarity_top_k: int = 1,
        openai_api_key: str = "",
    ) -> None:
        """
        Initialize the WerewolfAgent.

        Args:
            name (str): Name of the agent.
            sys_prompt (str): System prompt providing role context and instructions.
            model_config_name (str): Name of the model configuration, indicating which LLM will be used.
            reflect_before_vectorstore (bool): Whether to summarize or reflect on memory before retrieving vectorstore data.
            similarity_top_k (int, optional): The number of most similar messages/summaries to retrieve from the vectorstore.
            openai_api_key (str, optional): API key for OpenAI integration.
            model: The underlying LLM for generating responses.
            parser (ParserBase): Parser responsible for structuring, filtering, and formatting model outputs.
        """
        super().__init__(
            name=name,
            sys_prompt=sys_prompt,
            model_config_name=model_config_name,
        )

        self.parser = None
        self.reflect_before_vectorstore = reflect_before_vectorstore
        self.similarity_top_k = similarity_top_k
        self.model_config_name = model_config_name

        # Set OpenAI API key
        openai.api_key = openai_api_key

        # Initialize FAISS-based memory store
        if reflect_before_vectorstore:
            self.memory = ReflectiveVectorstoreMemory()
        else:
            self.memory = VectorstoreMemory()

    def set_parser(self, parser: ParserBase) -> None:
        """
        Set the parser for handling model outputs.

        This parser governs how responses are formatted, parsed,
        and stored (including any filtering of fields).
        """
        self.parser = parser

    def reply(self, x: Optional[Union[Msg, Sequence[Msg]]] = None) -> Msg:
        """
        Generate a response based on input messages, optionally retrieving context from memory.

        Args:
            x (Optional[Union[Msg, Sequence[Msg]]], optional):
                The input message(s) to process.

        Returns:
            Msg:
                The assistant's structured output, including any metadata defined by the parser.
        """ 
        query = Queries.werewolf_discussion_query

        # Retrieve relevant messages from memory
        if self.reflect_before_vectorstore:
            summary_context = self.memory.get_relevant_summaries_context(query=query, top_k=self.similarity_top_k)

            log_vectorstore_retrieval(query, summary_context)

            # Prepare prompt with context from retrieved summaries
            prompt = self.model.format(
                Msg("system", self.sys_prompt, role="system"),
                Msg(name="system", role="system", content=f"Summary of relevant past conversations: {summary_context}"),
                x,
                Msg("system", self.parser.format_instruction, "system"),
            )
        else:
            relevant_messages = self.memory.get_relevant_messages(query=query, top_k=self.similarity_top_k)

            log_vectorstore_retrieval(query, convert_messages_to_string(relevant_messages))

            # Prepare prompt with retrieved messages similar to input message
            prompt = self.model.format(
                Msg("system", self.sys_prompt, role="system"),
                relevant_messages and Msg("system", f"Relevant messages from memory: {relevant_messages}", "system"),
                x,
                Msg("system", self.parser.format_instruction, "system"),
            )

        # Call the LLM
        raw_response = retry_with_fallback(lambda: generate_model_response(self.model_config_name, prompt))

        self.speak(raw_response.text)

        # Parse the raw response
        parsed_response = self.parser.parse(raw_response).parsed

        vectorstore_entry = Msg(
            self.name,
            content=json.dumps(parsed_response),
            role="assistant",
        )

        # Save the response to memory
        self.memory.add_message(message=vectorstore_entry)

        msg = Msg(
            self.name,
            content=self.parser.to_content(parsed_response),
            role="assistant",
            metadata=self.parser.to_metadata(parsed_response),
        )

        return msg

    def lawyer_judge_decision(self, game_state: dict, available_targets: list, x: Optional[Union[Msg, Sequence[Msg]]] = None) -> Msg:
        """
        Make an elimination decision by invoking two Lawyers and one Judge.

        Args:
            game_state (dict): The current state of the Werewolf game (e.g., active players, werewolves).
            x (Optional[Union[Msg, Sequence[Msg]]], optional): Optionally, messages to be recorded in memory before decision-making.

        Returns:
            Msg:
                The final decision from the Judge as a structured message.
        """
        query = Queries.werewolf_discussion_query 

        # Retrieve relevant messages from memory (summaries or direct messages)
        if self.reflect_before_vectorstore:
            summary_context = self.memory.get_relevant_summaries_context(query=query, top_k=self.similarity_top_k)
            
            relevant_messages = [Msg(name="system", role="system", content=f"Summary of relevant past conversations: {summary_context}")]

            log_vectorstore_retrieval(query, summary_context)
        else:
            assert isinstance(query, str), f"Expected a string, but got {type(query)}"
            relevant_messages = self.memory.get_relevant_messages(query=query, top_k=self.similarity_top_k)

            log_vectorstore_retrieval(query, convert_messages_to_string(relevant_messages))
            
        # Initialize the lawyers and judge
        lawyer_1 = Lawyer(
            f'{self.name} (Lawyer 1)',
            self.sys_prompt,
            self.model_config_name,
            self.parser,
            relevant_messages,              
            self.memory
        )

        lawyer_2 = Lawyer(
            f'{self.name} (Lawyer 2)',
            self.sys_prompt,
            self.model_config_name,
            self.parser,
            relevant_messages,
            self.memory
        )

        judge = Judge(
            f'{self.name} (Judge)',
            self.sys_prompt,
            self.model_config_name,
            self.parser,
            relevant_messages,
            self.memory
        )
        
        structured_logger.log("game_state", {
            "werewolves": game_state["werewolves"],
            "villagers": game_state["villagers"],
            "seer": game_state["seer"],
            "witch": game_state["witch"],
            "survivors": game_state["survivors"],
            "dead": game_state["dead"],
            "witch_healing_used": False,
            "witch_poison_used": False,
            "endgame": False,
            "winner": None
        })
    
        lawyer_1_argument = lawyer_1.argue(available_targets)
        available_targets.remove(lawyer_1_argument["target"])
        lawyer_2_argument = lawyer_2.argue(available_targets)

        raw_response = judge.decide(game_state, lawyer_1_argument, lawyer_2_argument)

        self.speak(raw_response.text)

        # Parse response
        res = self.parser.parse(raw_response)

        logger.info("===")
        logger.info(f"Lawyer 1 argument: {lawyer_1_argument}")
        logger.info(f"Lawyer 2 argument: {lawyer_2_argument}")
        logger.info(f"Judge decision: {res.parsed}")
        logger.info("===")

        structured_logger.log("argument_response", lawyer_1_argument)
        structured_logger.log("argument_response", lawyer_2_argument)
        structured_logger.log("judge_response", res.parsed)

        vectorstore_entry = Msg(
            self.name,
            content="\n".join([
                f"Argument 1: {lawyer_1_argument}",
                f"Argument 2: {lawyer_2_argument}",
                f"Decision: {res.parsed}"
            ]),
            role="assistant"
        )

        # Store final decision
        self.memory.add_message(message=vectorstore_entry)

        msg = Msg(
            self.name,
            content=self.parser.to_content(res.parsed),
            role="assistant",
            metadata=self.parser.to_metadata(res.parsed),
        )

        return msg

    def observe(self, x: Union[Msg, Sequence[Msg]], role: str = None) -> None:
        """
        Record incoming messages in memory without generating a reply.

        Args:
            x (Union[Msg, Sequence[Msg]]): The message(s) to be stored in memory for future context.
            role (`str`, optional): Role of the observer
        """
        if role == "seer" and x is not None:
            self.memory.add_message(message=x)

    def summarize_cycle(self, survivors: list, secondary_model: str, cycle_type: str = "day"):
        """
        Produce a summary of key events at the end of a day or night cycle.

        Args:
            survivors: (list): List of current survivors.
            secondary_model (str): Name of secondary model.
            cycle_type (str, optional): The type of cycle being summarized, either "day" or "night".
        """
        self.memory.summarize_cycle(survivors, secondary_model, cycle_type=cycle_type)

    def save_context(self, timestamp: str, game_num: int, player_role: str="werewolf"):
        """
        Save self.messages to a log file in the format:
        vectorstore/[timestamp]/[game_num]/[player_role].log

        Args:
            game_num (int): The game index.
            player_role (str): The role of the player.
        """
        
        self.memory.save_context(timestamp, game_num, player_role)

class NormalAgent(AgentBase):
    """
    A general-purpose agent that returns structured responses in JSON/dict format.

    Integrates with a FAISS vectorstore for context retrieval.
    """

    def __init__(
        self,
        name: str,
        sys_prompt: str,
        model_config_name: str,
        reflect_before_vectorstore: bool,
        similarity_top_k: int = 1,
        openai_api_key: str = "",
    ) -> None:
        """
        Initialize the NormalAgent.

        Args:
            name (str): Agent name.
            sys_prompt (str): The system prompt containing the agent’s role instructions.
            model_config_name (str): Indicates which model configuration the agent should use.
            reflect_before_vectorstore (bool): Flag to determine if reflective memory is used before vectorstore retrieval.
            similarity_top_k (int, optional): The number of top similar items from the vectorstore to include as context.
            openai_api_key (str, optional): API key for integrating with OpenAI services.
        """
        super().__init__(
            name=name,
            sys_prompt=sys_prompt,
            model_config_name=model_config_name,
        )

        self.parser = None
        openai.api_key = openai_api_key
        self.similarity_top_k = similarity_top_k
        self.reflect_before_vectorstore = reflect_before_vectorstore
        self.model_config_name = model_config_name
        self.memory = VectorstoreMemory()

    def set_parser(self, parser: ParserBase) -> None:
        """
        Configure the parser that dictates how outputs are formatted, parsed, and stored.

        By adjusting the parser, you can control the entire pipeline of formatting and extraction
        for the model's responses.
        """
        self.parser = parser

    def reply(self, x: Optional[Union[Msg, Sequence[Msg]]] = None) -> Msg:
        """
        Generate a response based on provided input messages, using stored context as needed.

        Args: 
            x (Optional[Union[Msg, Sequence[Msg]]], optional): The incoming user or system messages.

        Returns:
            Msg:
                A structured message including the assistant's content and any parsed metadata.
        """
        query = Queries.non_werewolf_discussion_query

        # Retrieve similar messages from memory
        relevant_messages = self.memory.get_relevant_messages(query=query, top_k=self.similarity_top_k)

        log_vectorstore_retrieval(query, convert_messages_to_string(relevant_messages))

        # Prepare the prompt with relevant context
        prompt = self.model.format(
            Msg("system", self.sys_prompt, role="system"),
            relevant_messages and Msg("system", f"Relevant messages from memory: {relevant_messages}", "system"),
            x,
            Msg("system", self.parser.format_instruction, "system"),
        )

        # Call the LLM
        raw_response = retry_with_fallback(
            lambda: generate_model_response(self.model_config_name, prompt)
        )

        self.speak(raw_response.text)

        # Parse the model's output
        parsed_response = self.parser.parse(raw_response).parsed

        vectorstore_entry = Msg(
            self.name,
            content=json.dumps(parsed_response),
            role="assistant"
        )

        # Save the model's response to memory
        self.memory.add_message(message=vectorstore_entry)

        msg = Msg(
            self.name,
            content=self.parser.to_content(parsed_response),
            role="assistant",
            metadata=self.parser.to_metadata(parsed_response),
        )
        
        return msg

    def observe(self, x: Union[Msg, Sequence[Msg]], role: str = None) -> None:
        """
        Record incoming messages in memory without generating a reply.

        Args:
            x (Union[Msg, Sequence[Msg]]): The message(s) to be stored in memory for future context.
            role (`str`, optional): Role of the observer
        """
        if role == "seer" and x is not None:
            self.memory.add_message(message=x)

    def save_context(self, timestamp: str, game_num: int, player_role: str):
        """
        Save self.messages to a log file in the format:
        vectorstore/[timestamp]/[game_num]/[player_role].log

        Args:
            game_num (int): The game index.
            player_role (str): The role of the player.
        """
        
        self.memory.save_context(timestamp, game_num, player_role)


# Register your custom classes
agentscope.agents.NormalAgent = NormalAgent
agentscope.agents.WerewolfAgent = WerewolfAgent
agentscope.agents.Lawyer = Lawyer
agentscope.agents.Judge = Judge

## Prompts and Parser (edit minimally)
Below are the prompts that control and run the Werwolf game. They are implemented using the built in AgentScope MarkdownJsonDictParser which allows us to generate responses in a dictionary format that is compatible with our game. Currently structured to obtain both the private thoughts of an Agent which remain to themselves and showcase their reasoning, and the words that they speak to other agents.

Since these prompts control the game logic itself, we should try to limit the prompt edits here to a minimum, but we can make edits to the parser to control the reasoning process a particular role goes through before making a decision, eg. instead of a thought field, we ask it to make ask it to provide a reason to vote for a particular agent, and a reason to not vote for a particular agent, then ask it to make a final decision based off those fields. Fields other than "thought" however should not be touched as they play a critical role to the control flow of the game.

More details on the AgentScope parsers can be found here https://doc.agentscope.io/build_tutorial/structured_output.html

In [240]:
class Prompts:
    """Prompts for werewolf game"""

    # Lawyer & Judge Architecture

    lawyer_prompt = """
        You are arguing FOR the decision to eliminate {}.
        Argue FOR eliminating the player in bullet-point format. Use at most 3 bullets.
    """
    judge_prompt = """
        You are a judge deciding which player should be eliminated in a game of werewolf. Your goal is to maximize the player's chances of winning the game with forward-thinking reasoning.
        
        Game State:
        - Alive players: {}
        - Dead players: {}
        - Werewolves: {}

        The two lawyers have made their arguments:
        - **Lawyer 1** wants to eliminate {}: "{}"
        - **Lawyer 2** wants to eliminate {}: "{}"

        Make your final decision: Which player should be eliminated between the 2? Justify your reasoning briefly.
    """

    rules = """
        Player roles: In werewolf game, players are divided into two werewolves, two villagers, one seer and one witch. Note only werewolves know who are their teammates.
        - Werewolves: They know their teammates' identities and attempt to eliminate a villager each night while trying to remain undetected.
        - Villagers: They do not know who the werewolves are and must work together during the day to deduce who the werewolves might be and vote to eliminate them.
        - Seer: A villager with the ability to learn the true identity of one player each night. This role is crucial for the villagers to gain information.
        - Witch: A character who has a one-time ability to save a player from being eliminated at night (sometimes this is a potion of life) and a one-time ability to eliminate a player at night (a potion of death).

        Game Rule: The game is consisted of two phases: night phase and day phase. The two phases are repeated until werewolf or villager win the game.
        1. Night Phase: During the night, the werewolves discuss and vote for a player to eliminate. Special roles also perform their actions at this time (e.g., the Seer chooses a player to learn their role, the witch chooses a decide if save the player).
        2. Day Phase: During the day, all surviving players discuss who they suspect might be a werewolf. No one reveals their role unless it serves a strategic purpose. After the discussion, a vote is taken, and the player with the most votes is \"lynched\" or eliminated from the game.

        Victory Condition: For werewolves, they win the game if the number of werewolves is equal to or greater than the number of remaining villagers. For villagers, they win if they identify and eliminate all of the werewolves in the group.

        Constraints:
        1. Your response should be in the first person.
        2. This is a conversational game. You should response only based on the conversation history and your strategy.
    """
    
    to_wolves = (
        "{}, if you are the only werewolf, eliminate a player. Otherwise, "
        "discuss with your teammates and reach an agreement."
    )

    wolves_discuss_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "what you speak",
            "finish_discussion": "whether the discussion reached an agreement or not (true/false)",
        },
        required_keys=["thought", "speak", "finish_discussion"],
        keys_to_memory="speak",
        keys_to_content="speak",
        keys_to_metadata=["finish_discussion"],
    )

    to_wolves_vote = "Which player do you vote to kill?"

    wolves_vote_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "vote": "player_name",
        },
        required_keys=["thought", "vote"],
        keys_to_memory="vote",
        keys_to_content="vote",
    )

    to_wolves_res = "The player with the most votes is {}."

    to_witch_resurrect = (
        "{witch_name}, you're the witch. Tonight {dead_name} is eliminated. "
        "Would you like to resurrect {dead_name}?"
    )

    to_witch_resurrect_no = "The witch has chosen not to resurrect the player."
    to_witch_resurrect_yes = "The witch has chosen to resurrect the player."

    witch_resurrect_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "whether to resurrect the player and the reason",
            "resurrect": "whether to resurrect the player or not (true/false)",
        },
        required_keys=["thought", "speak", "resurrect"],
        keys_to_memory="speak",
        keys_to_content="speak",
        keys_to_metadata=["resurrect"],
    )

    to_witch_poison = (
        "Would you like to eliminate one player? If yes, "
        "specify the player_name."
    )

    witch_poison_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "what you speak",
            "eliminate": "whether to eliminate a player or not (true/false)",
        },
        required_keys=["thought", "speak", "eliminate"],
        keys_to_memory="speak",
        keys_to_content="speak",
        keys_to_metadata=["eliminate"],
    )

    to_seer = (
        "{}, you're the seer. Which player in {} would you like to check "
        "tonight?"
    )

    seer_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "player_name",
        },
        required_keys=["thought", "speak"],
        keys_to_memory="speak",
        keys_to_content="speak",
    )

    to_seer_result = "Okay, the role of {} is a {}."

    to_all_danger = (
        "The day is coming, all the players open your eyes. Last night, "
        "the following player(s) has been eliminated: {}."
    )

    to_all_peace = (
        "The day is coming, all the players open your eyes. Last night is "
        "peaceful, no player is eliminated."
    )

    to_all_discuss = (
        "Now the alive players are {}. Given the game rules and your role, "
        "based on the situation and the information you gain, to vote a "
        "player eliminated among alive players and to win the game, what do "
        "you want to say to others? You can decide whether to reveal your "
        "role."
    )

    survivors_discuss_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "what you speak",
        },
        required_keys=["thought", "speak"],
        keys_to_memory="speak",
        keys_to_content="speak",
    )

    survivors_vote_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "vote": "player_name",
        },
        required_keys=["thought", "vote"],
        keys_to_memory="vote",
        keys_to_content="vote",
    )

    to_all_vote = (
        "Given the game rules and your role, based on the situation and the"
        " information you gain, to win the game, it's time to vote one player"
        " eliminated among the alive players. Which player do you vote to "
        "kill?"
    )

    to_all_res = "{} has been voted out."

    to_all_wolf_win = (
        "The werewolves have prevailed and taken over the village. Better "
        "luck next time!"
    )

    to_all_village_win = (
        "The game is over. The werewolves have been defeated, and the village "
        "is safe once again!"
    )

    to_all_continue = "The game goes on."

    
# Moderator message function
HostMsg = partial(Msg, name="Moderator", role="assistant", echo=True)
QueryMsg = partial(Msg, name="Query", role="user", echo=False)

# Vectorstore Queries

In [241]:
class Queries:
    """Queries for the vectorstore based on certain points in game"""

    # Retrieves discussions about suspicions of non-werewolves to help identify potential enemies.
    werewolf_discussion_query = "Discussions related to villagers, seer, witch."

    # Retrieves discussions focused on identifying werewolves.
    non_werewolf_discussion_query = "Discussions related to werewolves."

## Game Initialization (edit here)
To initialize the agents, you must define their parameters and settings in the config objects that are passed in for initialization. There is a model config, which defines the base foundational model being used, and an agent config, which defines each of the agents being used in the game, including which model their using, their name, and what type of Agent they are (based off the agent classes we defined earlier). 

Pay particular attention to the system prompt, this is what defines the rules of the game to the agent and gives them the role and what their responsibilities are, we could perhaps do some prompt engineering with that.

Also we can play around with the settings of the game, eg. max rounds, how many werewolves we have, etc. Just make sure to update the roles, witch, seer objects below accordingly.

In [242]:
# Load API keys from .env file
load_dotenv()

# Define the API keys
openai_key = os.getenv("OPENAI_API_KEY")

## Utility Functions (don't edit)
Functions to check and update game state throughout

In [243]:
def retry_with_fallback(func: Callable[..., Any], *args: Any, **kwargs: Any) -> Any:
    """
    Tries to execute the given function. If it fails, waits 5 seconds and retries once.
    If it fails again, exits the script.

    Args:
        func (Callable[..., Any]): The function to execute.
        *args (Any): Positional arguments to pass to the function.
        **kwargs (Any): Keyword arguments to pass to the function.

    Returns:
        Any: The result of the function if successful.
    """
    try:
        return func(*args, **kwargs)  # Try executing the function
    except Exception as e:
        print(f"Error: {e}. Retrying in 5 seconds...")
        time.sleep(5)  # Wait 5 seconds before retrying

        try:
            return func(*args, **kwargs)  # Try again
        except Exception as e:
            print(f"Error: {e}. Failed again. Stopping execution.")
            sys.exit(1)  # Exit the script if it fails twice

def load_json_response(response: str) -> dict:
    """Converts a json response to a python dictionary"""
    return json.loads(response.replace("```json", "").replace("```", "").strip())

def convert_messages_to_string(relevant_messages: list) -> str:
    """Takes a list of messages, and converts them to a string separated by new line"""
    return '\n'.join([f"{msg.name}: {msg.content}" for msg in relevant_messages])

def majority_vote(votes: list) -> Any:
    """Given a list of votes, return the name with the highest frequency."""
    votes_valid = [v for v in votes if v != "Abstain"]
    if not votes_valid:
        return "No Votes"
    unique_vals, counts = np.unique(votes_valid, return_counts=True)
    return unique_vals[np.argmax(counts)]

def update_alive_players(game_state: dict, survivors: list, wolves: list, dead_names):
    """
    Removes 'dead_names' from 'game_state["survivors"]', updates game_state["dead"],
    and returns updated survivors and wolves lists.
    """
    if not isinstance(dead_names, list):
        dead_names = [dead_names]
    for d in dead_names:
        if d in game_state["survivors"]:
            game_state["survivors"].remove(d)
        if d not in game_state["dead"]:
            game_state["dead"].append(d)

    # Rebuild survivors / wolves lists
    new_survivors = [s for s in survivors if s.name in game_state["survivors"]]
    new_wolves = [w for w in wolves if w.name in game_state["survivors"]]
    return new_survivors, new_wolves

def check_winning(game_state: dict, survivors: list, wolves: list, host: str) -> bool:
    """
    If #werewolves * 2 >= #survivors => werewolves instantly win.
    If all werewolves are dead => villagers instantly win.
    Otherwise => game continues.
    """
    if len(wolves) * 2 >= len(survivors):
        msg = Msg(host, Prompts.to_all_wolf_win, role="assistant")
        game_state["endgame"] = True
        game_state["winner"] = "werewolves"
        log_message(msg)
        return True
    if survivors and not wolves:
        msg = Msg(host, Prompts.to_all_village_win, role="assistant")
        game_state["endgame"] = True
        game_state["winner"] = "villagers"
        log_message(msg)
        return True
    return False

def generate_log_filepath(basepath: str, 
                          reflect_before_vectorstore: bool, 
                          random_seed: int, 
                          timestamp: str, 
                          game_num: int) -> str:
    """Generate log file path with EST timestamp."""
    return f"{basepath}/{'reflect' if reflect_before_vectorstore else 'noreflect'}/{random_seed}/{timestamp}/courtroom_raw_game{game_num}.log"

def save_results(win_rate_filepath: str, row: dict):
    """
    Append a row to a CSV file. If doesn't exist, create with headers.

    row = {
      "raw_log_filepath": <str>,
      "custom_agent_won": <bool>,
    }
    """
    file_exists = os.path.isfile(win_rate_filepath)
    with open(win_rate_filepath, "a", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["raw_log_filepath", "custom_agent_won"])
        if not file_exists:
            w.writeheader()
        w.writerow(row)
    print(f"Row added to {win_rate_filepath}: {row}")

def custom_agent_won(game_state: dict, alive_agents: list, wolf_agents: list):
    """Records whether custom agent won (even if game didn't complete)"""

    # If game is completed, check if werewolves have won
    if game_state["endgame"]:
        if game_state["winner"] == "werewolves":
            return True 
        return False
    
    # If game not completed, then custom agent wins because it is still alive and eliminated another player
    return True

def get_available_targets(survivors: list, wolves: list):
    """Returns list of currently alive non-werewolves"""
    return [survivor.name for survivor in survivors if survivor not in wolves]

## Running a Game (don't edit)
Following is a function that game through the various Night and Day phases, taking different actions for each agent based on their roles. Multi agent functionality and communication is facilitated through the AgentScope Pipeline and MsgHub, more detailed documentation found here https://doc.agentscope.io/build_api/agentscope.pipelines.pipeline.html#module-agentscope.pipelines.pipeline and https://doc.agentscope.io/build_api/agentscope.msghub.html#module-agentscope.msghub

In [244]:
def run_game(
        secondary_model,
        max_days_per_game,
        reflect_before_vectorstore,
        max_werewolf_discussion_round,
        wolves,
        seer,
        witch,
        roles,
        survivors,
        game_state
    ):
    """
    Runs a single game, storing logs in game_log. This example merges both
    day/night flow, werewolf talk, witch usage, seer usage, etc. Then
    saves logs and results.
    """

    for day_i in range(1, max_days_per_game + 1):

        structured_logger.log("phase", "Night Time")

        # 1) Night Phase: Werewolves discussion
        hint = HostMsg(content=Prompts.to_wolves.format(n2s(wolves)))
        log_message(hint)
        with msghub(wolves, announcement=hint) as hub:
            set_parsers(wolves, Prompts.wolves_discuss_parser)
            for r in range(max_werewolf_discussion_round):
                x = retry_with_fallback(lambda: sequentialpipeline(wolves))
                logger.info(f"Werewolves discussion: {x.content}")
                if x.metadata.get("finish_discussion", False):
                    break
            # Then vote
            set_parsers(wolves, Prompts.wolves_vote_parser)
            hint_vote = HostMsg(content=Prompts.to_wolves_vote)
            log_message(hint_vote)
            votes = [extract_name_and_id(wolf.lawyer_judge_decision(game_state, get_available_targets(survivors, wolves), hint_vote).content)[0] for wolf in wolves]

            voted_out = majority_vote(votes)
            dead_player = [voted_out]

            voted_out_hint = HostMsg(content=Prompts.to_wolves_res.format(voted_out))
            hub.broadcast(voted_out_hint)
            log_message(voted_out_hint)

        # 2) Witch Decision Night   
        healing_used_tonight = False
        if witch in survivors:
            if not game_state["witch_healing_used"]:
                hint = HostMsg(
                    content=Prompts.to_witch_resurrect.format_map(
                        {
                            "witch_name": witch.name,
                            "dead_name": dead_player[0],
                        },
                    ),
                )
                log_message(hint)
                set_parsers(witch, Prompts.witch_resurrect_parser)
                
                # Capture the witch's resurrection response and log it
                resurrection_response = retry_with_fallback(lambda: witch(hint))

                if resurrection_response.metadata.get("resurrect", False):
                    healing_used_tonight = True
                    dead_player.pop()
                    game_state["witch_healing_used"] = True
                    HostMsg(content=Prompts.to_witch_resurrect_yes)
                    log_message(HostMsg(content=Prompts.to_witch_resurrect_yes))
                else:
                    HostMsg(content=Prompts.to_witch_resurrect_no)
                    log_message(HostMsg(content=Prompts.to_witch_resurrect_no))

            if not game_state["witch_poison_used"] and not healing_used_tonight:
                set_parsers(witch, Prompts.witch_poison_parser)
                
                # Capture the witch's poison response and log it
                poison_response = retry_with_fallback(lambda: witch(HostMsg(content=Prompts.to_witch_poison)))
                logger.info(f"Witch Poison Response: {poison_response.content}")

                if poison_response.metadata.get("eliminate", False):
                    target_player = extract_name_and_id(poison_response.content)[0]
                    dead_player.append(target_player)
                    game_state["witch_poison_used"] = True
                    logger.info(f"The witch has chosen to poison {target_player}.")
                else:
                    logger.info("The witch has chosen not to poison a player.")

        # 3) Seer checks a role
        if seer in survivors:
            seer_hint = HostMsg(content=Prompts.to_seer.format(seer.name, n2s(survivors)))
            log_message(seer_hint)
            set_parsers(seer, Prompts.seer_parser)

            x = seer.reply(seer_hint) # Use seer hint as query
            logger.info(f"Seer response: {x.content}")
            pl, idx = extract_name_and_id(x.content)
            role_name = roles[idx] if idx < len(roles) else "villager"
            # Let seer quietly observe the result
            role_msg = HostMsg(content=Prompts.to_seer_result.format(pl, role_name))
            logger.info(f"Moderator: {role_msg.content}")
            seer.observe(role_msg, "seer")

        # 4) Update survivors after night
        survivors, wolves = update_alive_players(game_state, survivors, wolves, dead_player)
        if check_winning(game_state, survivors, wolves, "Moderator"):
            break

        # If reflecting, do a night reflection
        if reflect_before_vectorstore:
            for w in wolves:
                w.summarize_cycle(survivors, secondary_model)

        structured_logger.log("phase", "Day Time")

        # 5) Daytime discussion
        content = (
            Prompts.to_all_danger.format(n2s(dead_player))
            if dead_player
            else Prompts.to_all_peace
        )
        discuss_hints = [
            HostMsg(content=content),
            HostMsg(content=Prompts.to_all_discuss.format(n2s(survivors))),
        ]
        with msghub(survivors, announcement=discuss_hints) as hub:
            set_parsers(survivors, Prompts.survivors_discuss_parser)
            discussion_out = retry_with_fallback(lambda: sequentialpipeline(survivors)) 
            logger.info(f"Survivors Discussion: {discussion_out.content}")

            # Daytime vote
            set_parsers(survivors, Prompts.survivors_vote_parser)
            day_vote_hint = HostMsg(content=Prompts.to_all_vote)
            log_message(day_vote_hint)
            
            votes_day = [
                extract_name_and_id(wolf.lawyer_judge_decision(game_state, get_available_targets(survivors, wolves), day_vote_hint).content)[0] for wolf in wolves
            ]

            votes_day.extend([
                extract_name_and_id(_.reply(day_vote_hint).content)[0] for _ in survivors if _ not in wolves
            ])
            day_result = majority_vote(votes_day)
            day_result_out = HostMsg(content=Prompts.to_all_res.format(day_result))
            hub.broadcast(day_result_out)
            log_message(day_result_out)

            survivors, wolves = update_alive_players(game_state, survivors, wolves, day_result)
            if check_winning(game_state, survivors, wolves, "Moderator"):
                break

            # If reflecting, do a day reflection
            if reflect_before_vectorstore:
                for w in wolves:
                    w.summarize_cycle(survivors, secondary_model)

            hub.broadcast(HostMsg(content=Prompts.to_all_continue))
    
    return custom_agent_won(
            game_state,
            survivors,
            wolves
        )


In [245]:
def run_experiment(
    open_ai_key: str,
    basefilepath: str,
    max_days_per_game: int = 10,
    num_games: int = 10,
    reflect_before_vectorstore: bool = True,
    primary_model: str = "gpt-4o-mini",
    secondary_model: str="gpt-4o",
    max_werewolf_discussion_round: int = 3,
    similarity_top_k: int = 1,
    random_seed: int = 85
) -> None:  
    """
    Initialize and run multiple games of the Werewolf simulation.

    Arguments:
        open_ai_key (`str`):
            The OpenAI API key used for model calls.
        basefilepath (`str`):
            Base filepath to store logs
        max_days_per_game (`int`, defaults to `3`):
            Maximum number of day/night cycles per game.
        num_games (`int`, defaults to `2`):
            Number of games to simulate.
        reflect_before_vectorstore (`bool`, defaults to `False`):
            Whether to use reflective memory before vectorstore retrieval.
        primary_model (`str`, defaults to `"gpt-4o-mini"`):
            Name of the model used for fast processing.
        max_werewolf_discussion_round (`int`, defaults to `3`):
            Maximum number of discussion rounds for werewolves.
        similarity_top_k (`int`, defaults to `1`):
            Number of messages or summaries retrieved from memory.
        random_seed (`int`, defaults to `85`):
            Number used to generate random seed

    Returns:
        `None`
    """
    global structured_logger

    # Get current timestamp
    timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Set random seed for reproducibility
    random.seed(random_seed)
    
    # Set openai api key
    openai.api_key = open_ai_key

    # Define model configs and settings
    model_configs = [
        {
            "model_type": "openai_chat",
            "config_name": primary_model,
            "model_name": primary_model,
            "api_key": open_ai_key,
            "generate_args": {
                "temperature": 1,
            }
        }
    ]

    # Define the config settings for each agent involved
    agent_configs = [
        {
            "class": "WerewolfAgent",
            "args": {
                "name": "Player1",
                "sys_prompt": f"""
                    You are a strategic decision-maker playing a game of werewolf.

                    You are Player1 and there are totally 4 players, named Player1, Player2, Player3, Player4.
                    
                    Rules of the game:

                    {Prompts.rules}
                    
                    You are playing werewolf in this game.
                """,
                "model_config_name": primary_model,
                "reflect_before_vectorstore": reflect_before_vectorstore,
                "similarity_top_k": similarity_top_k,
                "openai_api_key": open_ai_key
            }
        },
        {
            "class": "NormalAgent",
            "args": {
                "name": "Player2",
                "sys_prompt": f""" 
                    You are a strategic decision-maker playing a game of werewolf.

                    You are Player2 and there are totally 4 players, named Player1, Player2, Player3, Player4.

                    Rules of the game: 
                
                    {Prompts.rules}
                    
                    You are playing villager in this game.
                """,
                "model_config_name": primary_model,
                "reflect_before_vectorstore": reflect_before_vectorstore,
                "similarity_top_k": similarity_top_k,
                "openai_api_key": open_ai_key
            }
        },
        {
            "class": "NormalAgent",
            "args": {
                "name": "Player3",
                "sys_prompt": f""" 
                    You are a strategic decision-maker playing a game of werewolf.

                    You are Player3 and there are totally 4 players, named Player1, Player2, Player3, Player4.

                    Rules of the game:

                    {Prompts.rules}
                    
                    You are playing seer in this game.
                """,
                "model_config_name": primary_model,
                "reflect_before_vectorstore": reflect_before_vectorstore,
                "similarity_top_k": similarity_top_k,
                "openai_api_key": open_ai_key
            }
        },
        {
            "class": "NormalAgent",
            "args": {
                "name": "Player4",
                "sys_prompt": f""" 
                    You are a strategic decision-maker playing a game of werewolf.
                    
                    You are Player4 and there are totally 4 players, named Player1, Player2, Player3, Player4.
                    
                    Rules of the game:

                    {Prompts.rules}
                    
                    You are playing witch in this game.
                """,
                "model_config_name": primary_model,
                "reflect_before_vectorstore": reflect_before_vectorstore,
                "similarity_top_k": 1,
                "openai_api_key": open_ai_key
            }
        }
    ]

    for game_num in range(1, num_games + 1):
        tsv_log_filepath = f"{basefilepath}/{'reflect' if reflect_before_vectorstore else 'noreflect'}/{random_seed}/{timestamp_str}/courtroom_raw_game{game_num}.tsv"
        os.makedirs(os.path.dirname(tsv_log_filepath), exist_ok=True)
        structured_logger = StructuredLogger(tsv_log_filepath)

        # Read model and agent configs, and initialize agents automatically
        survivors = agentscope.init(
            model_configs=model_configs,
            agent_configs=agent_configs,
            project="Werewolf",
        )

        # Get player roles
        roles = ["werewolf", "villager", "seer", "witch"]
        wolves, witch, seer, villagers = [survivors[0]], survivors[-1], survivors[-2], [survivors[-3]]

        # Initialize game state
        game_state = {
            "werewolves": [player.name for player in wolves],
            "villagers": [player.name for player in survivors if player not in wolves],
            "seer": [seer.name],
            "witch": [witch.name],
            "survivors": [player.name for player in survivors],
            "dead": [],
            "witch_healing_used": False,
            "witch_poison_used": False,
            "endgame": False,
            "winner": None
        }

        current_log_path = generate_log_filepath(basefilepath, reflect_before_vectorstore, random_seed, timestamp_str, game_num)
        change_log_file(current_log_path)

        config = {
            "max_days_per_game": max_days_per_game,
            "primary_model": primary_model, 
            "secondary_model": secondary_model,
            "max_days_per_game": max_days_per_game, 
            "num_games": num_games,
            "reflect_before_vectorstore": reflect_before_vectorstore, 
            "max_werewolf_discussion_round": max_werewolf_discussion_round, 
            "similarity_top_k": similarity_top_k
        }
        logger.info(f"Starting game {game_num} with config: {config}")
        structured_logger.log("game_init", config)

        custom_agent_won = run_game(
            secondary_model,
            max_days_per_game,
            reflect_before_vectorstore,
            max_werewolf_discussion_round,
            wolves,
            seer,
            witch,
            roles,
            survivors,
            game_state
        )

        wolves[0].save_context(timestamp_str, game_num)
        witch.save_context(timestamp_str, game_num, "witch")
        seer.save_context(timestamp_str, game_num, "seer")
        villagers[0].save_context(timestamp_str, game_num, "villager")


        row = {
            "raw_log_filepath": current_log_path,
            "custom_agent_won": custom_agent_won
        }
        save_results(f"{basefilepath}/{'reflect' if reflect_before_vectorstore else 'noreflect'}/{random_seed}/{timestamp_str}/courtroom_win_rate.csv", row)

In [ ]:
run_experiment(
    open_ai_key=openai_key,
    basefilepath="logs/courtroom",
    primary_model = "gpt-4o-mini",
    secondary_model ="gpt-4o",
    reflect_before_vectorstore=False,
    max_days_per_game=5,
    num_games=20,
    similarity_top_k=2,
    random_seed=42
)

2025-02-28 15:17:03 - INFO - Starting game 1 with config: {'max_days_per_game': 5, 'primary_model': 'gpt-4o-mini', 'secondary_model': 'gpt-4o', 'num_games': 20, 'reflect_before_vectorstore': False, 'max_werewolf_discussion_round': 3, 'similarity_top_k': 2}
2025-02-28 15:17:03 - INFO - Moderator: Player1, if you are the only werewolf, eliminate a player. Otherwise, discuss with your teammates and reach an agreement.
2025-02-28 15:17:03 - INFO - ===
2025-02-28 15:17:03 - INFO - Query: Discussions related to villagers, seer, witch.
2025-02-28 15:17:03 - INFO - Retrieved similar documents: 
2025-02-28 15:17:03 - INFO - ===


Moderator: Player1, if you are the only werewolf, eliminate a player. Otherwise, discuss with your teammates and reach an agreement.


2025-02-28 15:17:04 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Player1: ```json
{"thought": "I need to stay under the radar while working with my werewolf partner. It's important to sow confusion among the villagers and avoid suspicion. I should steer discussions away from me and subtly accuse others.", "speak": "I think we should really consider Player3's actions last night. They seem suspicious to me, and we need to be cautious.", "finish_discussion": false}
```


2025-02-28 15:17:05 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-02-28 15:17:05 - INFO - Werewolves discussion: I think we should really consider Player3's actions last night. They seem suspicious to me, and we need to be cautious.
2025-02-28 15:17:06 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-02-28 15:17:06 - INFO - ===
2025-02-28 15:17:06 - INFO - Query: Discussions related to villagers, seer, witch.
2025-02-28 15:17:06 - INFO - Retrieved similar documents: Player1: {"thought": "I need to stay under the radar while working with my werewolf partner. It's important to sow confusion among the villagers and avoid suspicion. I should steer discussions away from me and subtly accuse others.", "speak": "I think we should really consider Player3's actions last night. They seem suspicious to me, and we need to be cautious.", "finish_discussion": false}
Player1: {"thought": "I need to stay under the radar w

...

Player3: ```json
{"thought": "I need to gather information about the players to identify the werewolves. I suspect Player1 might be a werewolf based on their previous behavior, so I will check Player1 tonight.","speak": "Player1"}
```


2025-02-28 15:29:08 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-02-28 15:29:09 - INFO - Seer response: Player1
2025-02-28 15:29:09 - INFO - Moderator: Okay, the role of Player1 is a werewolf.


Moderator: Okay, the role of Player1 is a werewolf.


2025-02-28 15:29:09 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-02-28 15:29:09 - INFO - Moderator: The werewolves have prevailed and taken over the village. Better luck next time!


Messages saved to vectorstore/courtroom/20250228_151703/20/werewolf.log
Messages saved to vectorstore/courtroom/20250228_151703/20/witch.log
Messages saved to vectorstore/courtroom/20250228_151703/20/seer.log
Messages saved to vectorstore/courtroom/20250228_151703/20/villager.log
Row added to logs/courtroom/noreflect/43/20250228_151703/courtroom_win_rate.csv: {'raw_log_filepath': 'logs/courtroom/noreflect/43/20250228_151703/courtroom_raw_game20.log', 'custom_agent_won': True}
